<a href="https://colab.research.google.com/github/carlocamilloni/Structural-Bioinformatics/blob/main/Notebooks/lab03_MD_enhanced_martini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Enhanced Sampling and Simplified Models in MD


In this lab experience you will perform two different simulations covering some more advanced topic. Namely:


*   A metadynamics simulation of alanine dipeptide;
*   A self-assembly and equilibration simulation of a lipid bilayer using the Martini force-field.




## Preliminary Operations

To perform the above tasks you first need to install some software, this will require to reinitialise the system.

The first step is to install the conda python package manager

In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:30
🔁 Restarting kernel...


Now we can install GROMACS (a molecular dynamics package) and PLUMED, that is a library to perform enhanced sampling techniques in combination with multiple MD packages, including gromacs (www.plumed.org). Be patient, the next step can take 5 minutes.

In [2]:
!conda install --strict-channel-priority -c plumed/label/masterclass -c conda-forge gromacs

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package

Now you are ready to start, so

1.   Make a copy of this notebook (File -> Save a copy in Drive)
2.   Mount your google drive
3.   Make a new folder for this task using the menus on the left side, for example in /drive/MyDrive/Structural_Bioinformatics make a folder named Task3
4.   In the Task3 folder make a folder named MTD




In [3]:
# mount google drive  
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [5]:
# set the path to your local task folder
mtd_path='/content/drive/MyDrive/Structural_Bioinformatics/Task3/MTD'
%cd {mtd_path}

/content/drive/MyDrive/Structural_Bioinformatics/Task3/MTD


In [7]:
# test the gromacs executable
!gmx mdrun -h

                      :-) GROMACS - gmx mdrun, 2019.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Ber

In [8]:
# make a copy of the GitHub structural bioinformatics repository
# this containts usefull input files for runnign the simulatoins
!git clone https://github.com/carlocamilloni/Structural-Bioinformatics.git ../sb --depth 1

Cloning into '../sb'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 49 (delta 6), reused 41 (delta 4), pack-reused 0
Unpacking objects: 100% (49/49), done.
Checking out files: 100% (38/38), done.


##Metadynamics simulations on Alanine Dipetide in Vacuum

In https://github.com/carlocamilloni/Structural-Bioinformatics/blob/main/Notebooks/lab02_MD.ipynb you performed a conventional MD simulation of alanine dipeptide in vacuum. Now you can repeat the same simulation but using Metadynamics to enhance the sampling of conformational space with respect to some conformational parameters, in particular we will use the phi and psi angles.

###System preparation

First of all let's prepare the system as you already done before:

In [13]:
%cd {mtd_path}
%cp ../sb/Data/md/inputs/aladp.pdb .
%cp ../sb/Data/md/mdps/0-em-steep.mdp em.mdp
%cp ../sb/Data/md/mdps/run.mdp .

/content/drive/MyDrive/Structural_Bioinformatics/Task3/MTD


In [12]:
# select force-field
!gmx pdb2gmx -f aladp.pdb -water none -ff amber03
# set the box
!gmx editconf -f conf.gro -bt cubic -d 1 -o boxed.gro
# energy minimisation
!gmx grompp -f em.mdp -c boxed.gro -p topol.top -o em.tpr
!gmx mdrun -deffnm em -v

                     :-) GROMACS - gmx pdb2gmx, 2019.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Be

In [17]:
# From the energy minised file we also make a reference PDB structure that we can use later
!gmx editconf -f em.gro -o reference.pdb

                     :-) GROMACS - gmx editconf, 2019.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, B

In [14]:
#now perform a preliminary 10 ns MD simulation, this will last less than 2 minutes
!gmx grompp -f run.mdp -c em.gro -p topol.top -o run.tpr
!gmx mdrun -deffnm run -v -ntomp 1

                      :-) GROMACS - gmx grompp, 2019.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Be

In [15]:
#Now you should have multiple files, in particular run.gro and run.xtc that are 
#the last frame and the trajectory from the MD simulation, respectively.
%ls

 aladp.pdb        em.edr   em.trr           run.edr   run.xtc
 boxed.gro        em.gro   mdout.mdp        run.gro   topol.top
'#boxed.gro.1#'   em.log   posre.itp        run.log  '#topol.top.1#'
 conf.gro         em.mdp  '#posre.itp.1#'   run.mdp
'#conf.gro.1#'    em.tpr   run.cpt          run.tpr


### MTD preparation

PLUMED allows both to analyze simulations trajectories by defining collective variables as well as then use CV based enhanced sampling techniques like Umbrella Sampling and Metadynamics to enhance the sampling of your MD. You can learn more on PLUMED syntax and features here: https://www.plumed.org/masterclass

Furthermore, you can find real-life simulations setups here: www.plumed-nest.org

As a first example let's analyse your MD simulation using plumed. To do so you need to write a plumed input file, that is a text file, including the definition of the quantity and the operation you want to perform. 

You can now copy an example file and see its content:

In [24]:
#copy the file in your mtd folder
%cp ../sb/Data/advanced/metad/plumed-cv.dat .
#print the file on screen
%cat plumed-cv.dat

# Activate MOLINFO functionalities that allow using syntax shortcuts
MOLINFO STRUCTURE=reference.pdb
# Compute the backbone dihedral angle phi, defined by atoms C-N-CA-C
phi: TORSION ATOMS=@phi-2
# Compute the backbone dihedral angle psi, defined by atoms N-CA-C-N
psi: TORSION ATOMS=@psi-2
# Print the two collective variables on COLVAR file every STRIDE steps
PRINT ARG=phi,psi FILE=COLVAR STRIDE=1


In [25]:
#analyse your simulation using plumed executable and your input file
!plumed driver --plumed plumed-cv.dat --mf_xtc run.xtc


DRIVER: Found molfile format trajectory xtc with name run.xtc
PLUMED: PLUMED is starting
PLUMED: Version: 2.8.0 (git: Unknown) compiled on Feb 23 2022 at 09:52:46
PLUMED: Please cite these papers when using PLUMED [1][2]
PLUMED: For further information see the PLUMED web page at http://www.plumed.org
PLUMED: Root: /usr/local/lib/plumed
PLUMED: For installed feature, see /usr/local/lib/plumed/src/config/config.txt
PLUMED: Molecular dynamics engine: driver
PLUMED: Precision of reals: 8
PLUMED: Running over 1 node
PLUMED: Number of threads: 1
PLUMED: Cache line size: 512
PLUMED: Number of atoms: 22
PLUMED: File suffix: 
PLUMED: FILE: plumed-cv.dat
PLUMED: Action MOLINFO
PLUMED:   with label @0
PLUMED:   pdb file named reference.pdb contains 1 chains 
PLUMED:   chain named   contains residues 1 to 3 and atoms 1 to 22 
PLUMED: Action TORSION
PLUMED:   with label phi
PLUMED:   between atoms 5 7 9 15
PLUMED:   using periodic boundary conditions
PLUMED: Action TORSION
PLUMED:   with label psi

In [28]:
# list files
%ls
# you should have a new COLVAR-CV.txt file

 aladp.pdb        em.edr   em.trr           run.cpt   run.xtc
 boxed.gro        em.gro   mdout.mdp        run.edr   topol.top
'#boxed.gro.1#'   em.log   plumed-cv.dat    run.gro  '#topol.top.1#'
 COLVAR-CV.txt    em.mdp   posre.itp        run.log
 conf.gro         em.pdb  '#posre.itp.1#'   run.mdp
'#conf.gro.1#'    em.tpr   reference.pdb    run.tpr


In [30]:
#let's have a look at it:
#this print on screen only the first tops lines
!head COLVAR-CV.txt
# the file has some header lines reporting the content of the file, in this
# case is three columns reporting time, phi and psi actions that are defined
# in your plumed-cv.dat input file
# then it tells you that phi and psi are defined in some range of values
# then you have the actual data (angles are in radians here)

#! FIELDS time phi psi
#! SET min_phi -pi
#! SET max_phi pi
#! SET min_psi -pi
#! SET max_psi pi
 0.000000 -2.435417 2.364389
 1.000000 -2.738451 2.378010
 2.000000 -1.555183 2.742969
 3.000000 -2.806969 -3.080113
 4.000000 -1.124978 0.287796
